In [1]:
!pip install pymysql

In [2]:
!pip install backtesting

In [3]:
!pip install pandas_ta

In [4]:
import os
import pandas as pd
#from strategy_utils import *
import time
from os.path import exists
import pandas_ta as ta
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Backtest
#import mysql.connector
#from mysql.connector import Error

/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [15]:
import pymysql

# database connection
connection = pymysql.connect(host="localhost", port=8889, user="root", passwd="root", database="quant")
cursor = connection.cursor()
# some other statements  with the help of cursor
connection.close()

OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)")

In [5]:
df = pd.read_csv('HINDALCO_1D.xlsx - HINDALCO.csv')
df

,datetime,close,high,low,open,volume,instrument
0,2014-01-24 00:00:00,114.00,115.35,113.00,113.15,5737135,HINDALCO
1,2014-01-27 00:00:00,111.10,112.70,109.30,112.00,8724577,HINDALCO
2,2014-01-28 00:00:00,113.80,115.00,109.75,110.00,4513345,HINDALCO
3,2014-01-29 00:00:00,111.75,114.75,111.15,114.50,4713458,HINDALCO
4,2014-01-30 00:00:00,108.10,110.70,107.60,110.20,5077231,HINDALCO
...,...,...,...,...,...,...,...
1210,2018-12-21 00:00:00,224.00,227.45,221.80,224.25,6933691,HINDALCO
1211,2018-12-24 00:00:00,217.85,224.00,216.95,224.00,4667022,HINDALCO
1212,2018-12-26 00:00:00,218.60,219.80,212.90,213.85,6554865,HINDALCO
1213,2018-12-27 00:00:00,220.60,223.90,219.60,221.40,7947144,HINDALCO


In [6]:
df = df.rename(columns={"open": "Open", "instrument":"instrument","close": "Close", "low": "Low", "high": "High", "volume": "Volume"})


In [7]:
df['Volume'].dtypes

dtype('int64')

In [8]:
# test suite
import unittest

class TestType(unittest.TestCase):
	# test function to test whether obj is instance of class
  def setUp(self):
      self.number = df['Volume']
      self.obj = df['instrument']
      self.dec = df['Open'],df['High'],df['Low'],df['Close']
  def test_assert_is_with_type(self):
      self.assertTrue(type(self.number),int64)
  def test_assert_is_with_type(self):
      self.assertTrue(type(self.dec),float64)
  def test_assert_is_with_type(self):
      self.assertTrue(type(self.obj),object)


if __name__ == '__main__':
	unittest.main(argv=['first-arg-is-ignored'],exit=False)



.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [9]:
def EMA(df_in, window):
    df = df_in.copy()
    #ema_series = df['Close'].ewm(span=ema1_len, min_periods=ema1_len).mean()
    ema_series = ta.ema(df['Close'], length=window)
    return ema_series

def EMA2(df_in, window):
    df = df_in.copy()
    #ema_series2 = df['Close'].ewm(span=ema2_len, min_periods=ema2_len).mean()
    ema_series2 = ta.ema(df['Close'], length=window)
    return ema_series2

In [10]:
class EmaCrossoverStrategy(Strategy):
    ema1_len = 10
    ema2_len = 50
    def init(self):
        super().init()
        # Add indicators
        self.ema1 = self.I(EMA, self.data.df, self.ema1_len)
        self.ema2 = self.I(EMA2, self.data.df, self.ema2_len)
    def next(self):
        super().init()
        price = self.data.Close[-1]
        # Long entry
        if (not self.position) and crossover(self.ema1, self.ema2):
            #  Buy
            self.position.close()
            self.buy()
        # Long exit
        elif self.position and crossover(self.ema2, self.ema1):
            # Close any existing long trades, and sell the asset
            self.position.close()
            self.sell()

In [11]:
#symbol, #data_file_map
def run_backtest(df):
    df = df
    #data_interval = "1H"
    #  Get data file
    #key = f"{symbol}_{data_interval}"
    #df = data_file_map[key]
    #if df is None or len(df) == 0:
        #print(f"Empty file: {symbol}, interval: {data_interval}")
    # If exclusive orders (each new order auto-closes previous orders/position),
    # cancel all non-contingent orders and close all open trades beforehand
    bt = Backtest(df, EmaCrossoverStrategy, cash=10000, commission=.00075, trade_on_close=True, exclusive_orders=True, hedging=False)
    stats = bt.run()
    print(stats)
    bt.plot()

In [13]:
if __name__ == '__main__':
    procs = []
    #load_all_data_files(symbols, data_interval_array)
    run_backtest(df)

/var/folders/rn/29jjrdlj1qd_s9ky5nyqtbnm0000gn/T/ipykernel_811/900399416.py:12: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, EmaCrossoverStrategy, cash=10000, commission=.00075, trade_on_close=True, exclusive_orders=True, hedging=False)


Start                                     0.0
End                                    1214.0
Duration                               1214.0
Exposure Time [%]                   35.226337
Equity Final [$]                          0.0
Equity Peak [$]                   18296.47935
Return [%]                             -100.0
Buy & Hold Return [%]               96.052632
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                   -9.835678
Max. Drawdown Duration                  710.0
Avg. Drawdown Duration              58.235294
# Trades                                  5.0
Win Rate [%]                             60.0
Best Trade [%]                       43.78545
Worst Trade [%]                   -101.256274
Avg. Trade [%]                    